In [9]:
import numpy as np
import pandas as pd
import os
import nltk
import re
from nltk.tokenize import word_tokenize

In [2]:
train_data = pd.read_csv('ratingsTrain.txt', header=0, delimiter='\t', quoting=3)
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [3]:
print(train_data['document'][123], train_data['label'][123])

별점10점가자 1


In [10]:
# nltk 데이터 다운로드
nltk.download('punkt')

# 사용자 정의 불용어 리스트
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한'])

def preprocessing(review, stop_words=None, remove_stopwords=False):
    if stop_words is None:
        stop_words = set()

    try:
        # 문장에서 한글만 뽑아냄
        review_text = re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", review)

        # nltk 라이브러리로 단어 단위로 분리
        word_review = word_tokenize(review_text)

        # 감탄사와 조사 제거
        if remove_stopwords:
            word_review = [token for token in word_review if token not in stop_words]

        return word_review

    except Exception as e:
        print(f"An error occurred during preprocessing: {e}")
        return None

# 예시 사용
review = "오늘은 정말 좋은 날씨네요!"
preprocessed_review = preprocessing(review, stop_words=stop_words, remove_stopwords=True)

if preprocessed_review is not None:
    print(preprocessed_review)

['오늘은', '정말', '좋은', '날씨네요']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\windows\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
import numpy as np
import pandas as pd
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize

# nltk 데이터 다운로드
nltk.download('punkt')

# 사용자 정의 불용어 리스트
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한'])

def preprocessing(review, stop_words=None, remove_stopwords=False):
    if stop_words is None:
        stop_words = set()

    try:
        # 문장에서 한글만 뽑아냄
        review_text = re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", review)

        # nltk 라이브러리로 단어 단위로 분리
        word_review = word_tokenize(review_text)

        # 감탄사와 조사 제거
        if remove_stopwords:
            word_review = [token for token in word_review if token not in stop_words]

        return word_review

    except Exception as e:
        print(f"An error occurred during preprocessing: {e}")
        return []

# 데이터 로딩
train_data = pd.read_csv('ratingsTrain.txt', header=0, delimiter='\t', quoting=3)

# 이상한 데이터 정제
clean_train_review = []
for review in train_data['document']:
    if isinstance(review, str):
        clean_train_review.append(preprocessing(review, stop_words=stop_words, remove_stopwords=True))
    else:
        clean_train_review.append([])

# Text to Number
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequence = tokenizer.texts_to_sequences(clean_train_review)

MAX_SEQUENCE_LENGTH = 8
train_inputs = pad_sequences(train_sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# train label 만들기
train_labels = np.array(train_data['label'])

# 10%는 Eval Data로 사용
TEST_SPLIT = 0.1
RNG_SEED = 13371447

# 데이터를 train과 eval로 분리
input_train, input_eval, label_train, label_eval = train_test_split(train_inputs, train_labels, test_size=TEST_SPLIT, random_state=RNG_SEED)

# 모델 정의
model = Sequential()
word_size = len(tokenizer.word_index) + 1
model.add(Embedding(word_size, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(input_train, label_train, epochs=5)

# 모델 평가
loss, accuracy = model.evaluate(input_eval, label_eval)
print(f"Evaluation - Loss: {loss}, Accuracy: {accuracy}")

# 예측 함수
def predict_review(sentence, model):
    # 테스트 문장을 전처리
    test_prepro = preprocessing(sentence, stop_words=stop_words, remove_stopwords=True)
    test_review = []
    test_review.append(test_prepro)
    
    # 전처리된 문장을 토큰화
    test_token = tokenizer.texts_to_sequences(test_review)
    test_seq = pad_sequences(test_token, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    ret = model.predict(test_seq)
    
    if ret > 0.5:
        sentiment = "긍정"
    elif ret < 0.5:
        sentiment = "부정"
    else:
        sentiment = "중립"
    
    return sentiment

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\windows\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch 1/5


c:\Users\windows\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


4219/4219 ━━━━━━━━━━━━━━━━━━━━ 565s 134ms/step - accuracy: 0.7194 - loss: 0.5286
Epoch 2/5
4219/4219 ━━━━━━━━━━━━━━━━━━━━ 568s 135ms/step - accuracy: 0.9467 - loss: 0.1655
Epoch 3/5
4219/4219 ━━━━━━━━━━━━━━━━━━━━ 546s 129ms/step - accuracy: 0.9839 - loss: 0.0458
Epoch 4/5
4219/4219 ━━━━━━━━━━━━━━━━━━━━ 550s 130ms/step - accuracy: 0.9890 - loss: 0.0272
Epoch 5/5
4219/4219 ━━━━━━━━━━━━━━━━━━━━ 565s 131ms/step - accuracy: 0.9909 - loss: 0.0211
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7701 - loss: 0.7586
Evaluation - Loss: 0.7625948786735535, Accuracy: 0.7660666704177856


In [18]:
# 테스트
print(predict_review('별점10점가자', model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
긍정


In [22]:
# 테스트
print(predict_review('별로다', model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
부정
